# Link Prediction

Um die Reichweite des Imperators im Sozialen Netzwerk zu vergrössern, verwenden wir eine Link Prediction. Somit wollen wir für nicht-verbundene Knoten eine passende Verbindung finden, um das Netzwerk zu vergrössern.

## Common Neighbours

Common Neighbours berechnet für zwei nich verbunde Knoten, wieviele gemeinsame nNachbarn existieren. 

Die Formel für gemeinsame Nachbarn lautet: 
$
\text{common\_neighbours}(X, Y) = \lvert N(X) \cap N(Y) \rvert
$

In [10]:
import json
import networkx as nx

with open('data/starwars-episode-1-interactions.json') as f:
    data_interactions = json.load(f)

# Graph erstellen: nutze direkt den 'name' aus den Node-Daten
G = nx.Graph()

for node in data_interactions['nodes']:
    # Verwende den 'name' als Knotenidentifikator
    G.add_node(node['name'])

for link in data_interactions['links']:
    src_id = link['source']
    tgt_id = link['target']
    # Falls source und target Indizes im nodes-Array sind:
    source_name = data_interactions['nodes'][src_id]['name']
    target_name = data_interactions['nodes'][tgt_id]['name']
    G.add_edge(source_name, target_name)


# Link-Prediction mit Common Neighbours
potential_links = []
nodes = list(G.nodes())
n = len(nodes)

for i in range(n):
    for j in range(i+1, n):
        u = nodes[i]
        v = nodes[j]
        if not G.has_edge(u, v):
            cn_count = len(list(nx.common_neighbors(G, u, v)))
            if cn_count > 0:
                potential_links.append((u, v, cn_count))

potential_links.sort(key=lambda x: x[2], reverse=True)

print("Top 10 Link Predictions basierend auf Common Neighbours mit Namen:")
for u, v, score in potential_links[:10]:
    print(f"{u} - {v}: gemeinsame Nachbarn = {score}")

print(G)

Top 10 Link Predictions basierend auf Common Neighbours mit Namen:
NUTE GUNRAY - JAR JAR: gemeinsame Nachbarn = 5
OBI-WAN - EMPEROR: gemeinsame Nachbarn = 5
OBI-WAN - SEBULBA: gemeinsame Nachbarn = 5
OBI-WAN - KITSTER: gemeinsame Nachbarn = 5
OBI-WAN - JABBA: gemeinsame Nachbarn = 5
OBI-WAN - RABE: gemeinsame Nachbarn = 5
CAPTAIN PANAKA - SHMI: gemeinsame Nachbarn = 5
SIO BIBBLE - BOSS NASS: gemeinsame Nachbarn = 5
SIO BIBBLE - ANAKIN: gemeinsame Nachbarn = 5
BOSS NASS - RIC OLIE: gemeinsame Nachbarn = 5
Graph with 37 nodes and 129 edges


## Jaccard Coefficient

Die Jaccarcd Coefficient Metrik ist eine Erweiterung der Common Neighbours Metrik.
Sie berücksichtigt nicht nur die Anzahl gemeinsamer Nachbarn zweier Knoten, sondern
setzt diese ins Verhältnis zu den Anzahl Nachbarn beider Knoten.

Die Formel für die Jaccard Coefficient lautet wie folgt: 

$
\text{jaccard\_coefficient}(X, Y) = \frac{\lvert N(X) \cap N(Y) \rvert}{\lvert N(X) \cup N(Y) \rvert}
$


In [ ]:
import json
import networkx as nx

with open('data/starwars-episode-1-interactions.json') as f:
    data_interactions = json.load(f)

# Erstelle den Graphen
G = nx.Graph()

for node in data_interactions['nodes']:
    G.add_node(node['name'])

# Falls 'source' und 'target' direkt die Namen enthalten, kannst du sie direkt nutzen.
# Falls es Indizes sind, musst du die Namen entsprechend aus data_interactions['nodes'] herauslesen.
for link in data_interactions['links']:
    src_id = link['source']
    tgt_id = link['target']
    source_name = data_interactions['nodes'][src_id]['name']
    target_name = data_interactions['nodes'][tgt_id]['name']
    G.add_edge(source_name, target_name)

# Jaccard-Koeffizienten für alle unverbundenen Paare berechnen
jaccard_scores = list(nx.jaccard_coefficient(G))

# jaccard_scores ist eine Liste von Tupeln (u, v, p) wobei p der Jaccard-Koeffizient ist
# Sortieren nach p absteigend
jaccard_scores.sort(key=lambda x: x[2], reverse=True)

# Top 10 ausgeben
print("Top 10 Link Predictions basierend auf Jaccard Coefficient:")
for u, v, score in jaccard_scores[:10]:
    print(f"{u} - {v}: Jaccard = {score:.4f}")

print(G)

Top 10 Link Predictions basierend auf Jaccard Coefficient:
SIO BIBBLE - BOSS NASS: Jaccard = 0.6250
BOSS NASS - RIC OLIE: Jaccard = 0.6250
JIRA - C-3PO: Jaccard = 0.6000
JIRA - WATTO: Jaccard = 0.6000
RUNE - GENERAL CEEL: Jaccard = 0.6000
SEBULBA - KITSTER: Jaccard = 0.5556
SHMI - BOSS NASS: Jaccard = 0.5556
SEBULBA - BOSS NASS: Jaccard = 0.5000
SEBULBA - JIRA: Jaccard = 0.5000
SEBULBA - FODE/BEED: Jaccard = 0.5000
Graph with 37 nodes and 129 edges


## Resource Allocatoin

Der Ansatz bei der Resource Allocation basiert darauf, dass ein Knoten X Ressourcen (z. B. Informationen) über gemeinsame Nachbarn an einen nicht direkt verbundenen Knoten Z weiterleiten kann. 

X sendet die Ressource an einen gemeinsamen Nachbarn Y, der sie gleichmäßig auf alle seine Nachbarn verteilt. Hat Y viele Nachbarn, erhält Z nur einen kleinen Anteil der Ressource.

Die Formel für Resource Allocaton lautet: 
$$
\text{resource\_allocation}(X, Y) = \sum_{u \in N(X) \cap N(Y)} \frac{1}{|N(u)|}
$$


In [11]:
import json
import networkx as nx

with open('data/starwars-episode-1-interactions.json') as f:
    data_interactions = json.load(f)

G = nx.Graph()

# für alle links in data_interactions einen Knoten hinzufügen
for node in data_interactions['nodes']:
    G.add_node(node['name'])

for link in data_interactions['links']:
    src_id = link['source']
    tgt_id = link['target']
    source_name = data_interactions['nodes'][src_id]['name']
    target_name = data_interactions['nodes'][tgt_id]['name']
    G.add_edge(source_name, target_name)


# Resource Allocation
ra_scores = list(nx.resource_allocation_index(G))

# ra_scores ist eine Liste von (u, v, p) Tupeln, wobei p der RA-Wert ist
ra_scores.sort(key=lambda x: x[2], reverse=True)

print("Top 10 Link Predictions basierend auf Resource Allocation:")
for u, v, score in ra_scores[:10]:
    print(f"{u} - {v}: RA = {score:.4f}")
print(G)

Top 10 Link Predictions basierend auf Resource Allocation:
RABE - OBI-WAN: RA = 0.5855
JAR JAR - NUTE GUNRAY: RA = 0.5826
NUTE GUNRAY - OBI-WAN: RA = 0.4417
RUNE - GENERAL CEEL: RA = 0.4409
TEY HOW - TC-14: RA = 0.4333
DOFINE - RUNE: RA = 0.4333
OBI-WAN - EMPEROR: RA = 0.4083
TEY HOW - DARTH MAUL: RA = 0.3500
TEY HOW - EMPEROR: RA = 0.3500
SIO BIBBLE - ANAKIN: RA = 0.3424
Graph with 37 nodes and 129 edges


## Ademic Adar

Diese Metrik berechnet sich fast genau gleich wie Resource Allocation. Der einzige
Unterschied ist, dass das Gewicht des Nenners abgeschwächt wird, indem da der Loga-
rithmus verwendet wird:

$$
\text{adamic\_adar}(X, Y) = \sum_{u \in N(X) \cap N(Y)} \frac{1}{\ln(|N(u)|)}
$$


In [12]:
import json
import networkx as nx

with open('data/starwars-episode-1-interactions.json') as f:
    data_interactions = json.load(f)

G = nx.Graph()

for node in data_interactions['nodes']:
    G.add_node(node['name'])

for link in data_interactions['links']:
    src_id = link['source']
    tgt_id = link['target']
    source_name = data_interactions['nodes'][src_id]['name']
    target_name = data_interactions['nodes'][tgt_id]['name']
    G.add_edge(source_name, target_name)


# Adamic-Adar
aa_scores = list(nx.adamic_adar_index(G))

# aa_scores ist eine Liste von (u, v, p) Tupeln, wobei p der Adamic-Adar-Wert ist
aa_scores.sort(key=lambda x: x[2], reverse=True)

print("Top 10 Link Predictions basierend auf Adamic-Adar:")
for u, v, score in aa_scores[:10]:
    print(f"{u} - {v}: Adamic-Adar = {score:.4f}")

print(G)

Top 10 Link Predictions basierend auf Adamic-Adar:
JAR JAR - NUTE GUNRAY: Adamic-Adar = 2.3159
RABE - OBI-WAN: Adamic-Adar = 2.3085
OBI-WAN - EMPEROR: Adamic-Adar = 1.9786
SIO BIBBLE - ANAKIN: Adamic-Adar = 1.8546
SIO BIBBLE - BOSS NASS: Adamic-Adar = 1.8546
BOSS NASS - RIC OLIE: Adamic-Adar = 1.8252
PADME - RIC OLIE: Adamic-Adar = 1.8252
SEBULBA - KITSTER: Adamic-Adar = 1.8140
SEBULBA - OBI-WAN: Adamic-Adar = 1.8140
KITSTER - JABBA: Adamic-Adar = 1.8140
Graph with 37 nodes and 129 edges


## Sind Links entstanden für 2. episode? 

In [13]:
# check if Jar Jar and Nute Gurnay have connection in 2. episode

import json
import networkx as nx

with open('data/starwars-episode-2-interactions.json') as f:
    data_interactions = json.load(f)

G = nx.Graph()

for node in data_interactions['nodes']:
    G.add_node(node['name'])

for link in data_interactions['links']:
    src_id = link['source']
    tgt_id = link['target']
    source_name = data_interactions['nodes'][src_id]['name']
    target_name = data_interactions['nodes'][tgt_id]['name']
    G.add_edge(source_name, target_name)

# check if Jar Jar and Nute Gurnay have connection in 2. episode
print("Jar Jar Binks - Nute Gunray connection in Episode 2:")
print(G.has_edge('Jar Jar Binks', 
                    'Nute Gunray'))

print(G)


Jar Jar Binks - Nute Gunray connection in Episode 2:
False
Graph with 32 nodes and 98 edges
